# MAGNETIC RESONANCE IMAGING (MRI)

**N.B.** This excerpt is taken from https://case.edu/med/neurology/NR/MRI%20Basics.htm 

Magnetic resonance imaging (MRI) is one of the most commonly used tests in neurology and neurosurgery. MRI provides exquisite detail of brain, spinal cord and vascular anatomy, and has the advantage of being able to visualize anatomy in all three planes: 
#### 1. axial
#### 2. sagittal &
#### 3. coronal 

(see the example image below). 

![](https://case.edu/med/neurology/NR/mri%20slices%20new.jpg "axial, sagittal and coronal plane")

# MRI IMAGING SEQUENCES
There are different types of MRI sequences. The most commons are the 
#### 1. T1-weighted (T1w)
#### 2. T2-weighted (T2w)
#### 3. Fluid Attenuated Inversion Recovery (FLAIR)

These sequences vary from each other in their TE (Time to Echo) and TR (Repetition Time) e.g. T1-weighted images are produced by using short TE and TR times while T2-weighted images are produced by using longer TE and TR times. 
T1-weighted imaging can also be performed while infusing Gadolinium (Gad). And that is the 4th type of sequence in this dataset:
#### 4. T1-weighted Gadolinium Post Contrast (T1wCE/T1Gd)

# Comparison of Different Sequences

N.B. image taken from this paper https://link.springer.com/article/10.1007/s10278-019-00282-4

![](https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs10278-019-00282-4/MediaObjects/10278_2019_282_Fig1_HTML.png)


**In a T1 weighted sequence:**
* fluid (CSF): black/dark - low signal intensity
* fat: white/light - high signal intensity
* grey matter: grey
* white matter: white-ish

**In a T2 weighted sequence:**
* fluid (CSF): white/bright
* fat: white/light
* grey matter: grey
* white matter: black/dark

FLAIR images appear similar to T1 (CSF is dark). The best way to tell the two apart is to look at the grey-white matter. T1 sequences will have grey matter being darker than white matter.

T1wGd images has bright signals in the blood vessels due to the Gadolinium.

In [ ]:
import glob
import pandas as pd
pd.set_option('display.max_colwidth', -1)  
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['animation.html'] = 'jshtml'
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras

import pydicom

In [ ]:
root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

df = pd.read_csv(root_dir+'train_labels.csv')

sns.countplot(data=df, x='MGMT_value')

# What is the 'MGMT_value'?

O[6]-methylguanine-DNA methyltransferase (MGMT) is a protein in cells, including tumour cells, that repairs damage to the cell’s DNA. For example, the damage caused by chemotherapy drugs to tumour cells. The more MGMT protein that the tumour produces, the less effective the chemotherapy drug is expected to be, as the protein will repair the damage to the tumour. Thus, determination of MGMT promoter methylation status in newly diagnosed GBM can influence treatment decision making.

In this dataset the MGMT promoter methylation status data is defined as a binary label (0: unmethylated, 1: methylated)



In [ ]:
# Add the full paths for each id for different types of sequences to the csv 
def full_ids(data):
    zeros = 5 - len(str(data))
    if zeros > 0:
        prefix = ''.join(['0' for i in range(zeros)])
    
    return prefix+str(data)
        

df['BraTS21ID_full'] = df['BraTS21ID'].apply(full_ids)

# Add all the paths to the df for easy access
df['flair'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/FLAIR/')
df['t1w'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1w/')
df['t1wce'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1wCE/')
df['t2w'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T2w/')
df

# DICOM
DICOM is the international standard to communicate and manage medical images and data. Its mission is to ensure the interoperability of systems used to produce, store, share, display, send, query, process, retrieve and print medical images, as well as to manage related workflows. 

Let's see how to read a dicom file with pydicom and what it looks like.

DICOM holds all MRI data such as: sequence type, MR acquisition type, patient orientation etc. along with the main MRI image.


In [ ]:
data = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-24.dcm')
data

In [ ]:
def get_image(data):
    '''
    Returns the image data as a numpy array.
    '''  
    if np.max(data.pixel_array)==0:
        img = data.pixel_array
    else:
        img = data.pixel_array/np.max(data.pixel_array)
        img = (img * 255).astype(np.uint8)
        
    return img

data = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-24.dcm')
img = get_image(data)
plt.imshow(img, cmap='gray')

# Display Sequence of MRIs as Animation
Each subfolder within individual case corresponds to a MRI Imaging Sequence. I am gonna use Matplotlib here to display the sequence of images as animation. This animations can also be saved in video format.

In [ ]:
def sorted_image_dirs(path: str):
    '''
    Sorts the list of image directories by image number in a path
    '''
    dirs = glob.glob(path+'*')
    dirs.sort(key=lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]))
    
    return dirs


def get_all_images(path: str):
    '''
    Returns a list of (non blank) images from a given path (of shape [non_blank_image_count, 512, 512])
    '''
    image_dirs = sorted_image_dirs(path)
    images = []
    
    for directory in image_dirs:
        data = pydicom.dcmread(directory)
        img = get_image(data)
        
        # Exclude the blank images
        if np.max(img)!=0:
            images.append(img)
        else:
            pass
    
    return images
    
def show_animation(images: list):
    '''
    Displays an animation from the list of images.
    
    set: matplotlib.rcParams['animation.html'] = 'jshtml'
    
    '''
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(images[0], cmap='gray')
    
    def animate_func(i):
        im.set_array(images[i])
        return [im]
    
    return matplotlib.animation.FuncAnimation(fig, animate_func, frames = len(images), interval = 20)

## Show FLAIR Sequence of a Patient as Images

In [ ]:
# A random patient - patient 10
patient = 10

flair_images = get_all_images(df['flair'][patient])
print('No of images:', len(flair_images))
print('MGMT: ', df['MGMT_value'][patient])

fig = plt.figure(figsize=(30,30))

c = 1
for image in flair_images:
    ax = fig.add_subplot(len(flair_images)//10+1, 10, c)
    ax.imshow(image, cmap='gray')
    c+=1
    
    plt.axis('off')
    
fig.tight_layout()

## Show Flair Sequence as Animation

In [ ]:
print('No of images:', len(flair_images))

flair_animation = show_animation(flair_images)
flair_animation
# flair_animation.save('./a.mp4')

## Show T1w Sequence as Animation

In [ ]:
t1w_images = get_all_images(df['t1w'][patient])
    
print('No of images:', len(t1w_images))
show_animation(t1w_images)

# MRI Plane
After reading a dicom file we can get all the information we need like patient orientation, pixel data (Each of the image is a 512x512 array) etc.
As mentioned at the begining of this notebook, MRI can be visualized in 3 different planes - Axial(Transverse), Coronal, Sagittal. The plane of an MR image relative to the patient's body can be calculated using the DICOM Image Orientation Patient tag. 

![](https://my-ms.org/images/mri_planes_gnu.jpg)

In [ ]:
# https://www.kaggle.com/davidbroberts/determining-mr-image-planes
def get_image_plane(data):
    '''
    Returns the MRI's plane from the dicom data.
    
    '''
    x1,y1,_,x2,y2,_ = [round(j) for j in data.ImageOrientationPatient]
    cords = [x1,y1,x2,y2]

    if cords == [1,0,0,0]:
        return 'coronal'
    if cords == [1,0,0,1]:
        return 'axial'
    if cords == [0,1,0,0]:
        return 'sagittal'

# Plot of Some Random MRIs with Their Respective Sequences, Planes & MGMT_values
As each of the sequence contains multiple images, I am gonna display the middle image of the sequence.

In [ ]:
def get_middle_image(path: str, plane=False):
    '''
    Returns the middle image from the path
    
    if plane=True returns the plane
    '''
    image_dirs = sorted_image_dirs(path)

    dicom = pydicom.dcmread(image_dirs[len(image_dirs)//2])
    img = get_image(dicom)
    plane = get_image_plane(dicom)
    
    if plane:
        return img, plane
    else:
        return img

In [ ]:
fig = plt.figure(figsize=(35,20))

seq_types = ['flair', 't1w', 't1wce', 't2w']

for i in range(20):
    
    patient = np.random.randint(low=0, high=len(df))
    seq_type = np.random.choice(seq_types)

    # path for the randomly selected image and sequence type
    seq_path = df[seq_type][patient]

    # Get the middle image and plane from the path
    img, plane = get_middle_image(seq_path, plane=True)
    
    patient_id, mgmt = df['BraTS21ID_full'][patient], df['MGMT_value'][patient]
    
    ax = fig.add_subplot(4,5,i+1)
    ax.imshow(img, cmap='gray')
    plt.title(f'ID: {patient_id}, MGMT_value: {mgmt}, Plane: {plane}, Seq_type: {seq_type}')
    plt.axis('off')

# The Planes of 4 type of Sequences for the Same Patient are Not the Same!

After plotting one image (the middle image) from each sequence for the first patient (ID 00000) we can see that, not all the axis are the same. Different sequences planes for one patient can be different.

In [ ]:
fig = plt.figure(figsize=(35,20))

seq_types = ['flair', 't1w', 't1wce', 't2w']

patient = 0  # The first patient

for i in range(4):
    
    seq_type = seq_types[i]
    seq_path = df[seq_type][patient]

    # Get the middle image and plane from the path
    img, plane = get_middle_image(seq_path, plane=True)
    
    patient_id, mgmt= df['BraTS21ID_full'][patient], df['MGMT_value'][patient]
    
    ax = fig.add_subplot(1,4,i+1)
    ax.imshow(img, cmap='gray')
    plt.title(f'ID: {patient_id}, MGMT_value: {mgmt}, Plane: {plane}, Seq_type: {seq_type}')
    plt.axis('off')

## Add the plane data for each sequence to the exisitng dataframe 

In [ ]:
seq_types = ['flair', 't1w', 't1wce', 't2w']

seq_axes = {'flair':[], 't1w':[], 't1wce':[], 't2w':[]}

for index in tqdm(range(len(df))):
    
    for seq in seq_types:
        
        seq_path = df[seq][index]

        # Get the middle image from the path
        img, plane = get_middle_image(seq_path, plane=True)
    
        seq_axes[seq].append(plane)
        
# Add the axes to the dataframe
df['flair_axis'] = seq_axes['flair']
df['t1w_axis'] = seq_axes['t1w']
df['t1wce_axis'] = seq_axes['t1wce']
df['t2w_axis'] = seq_axes['t2w']

# Print the axes with the respective patient ID
df[['BraTS21ID_full', 'flair_axis', 't1w_axis', 't1wce_axis', 't2w_axis']]

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(30,10))

sns.countplot(x='flair_axis', data=df, ax=ax[0])
sns.countplot(x='t1w_axis', data=df, ax=ax[1])
sns.countplot(x='t1wce_axis', data=df, ax=ax[2])
sns.countplot(x='t2w_axis', data=df, ax=ax[3])

plt.show()

# Important: 
In this [discussion](https://www.kaggle.com/c/rsna-miccai-brain-tumor-radiogenomic-classification/discussion/262046) a competition host has notified that there are some issues with these 3 cases - 

Patient IDs -  
* 00109 (FLAIR images are blank) 
* 00123 (T1w images are blank) 
* 00709 (FLAIR images are blank)

Hence these can be excluded.

In [ ]:
to_exclude = [109, 123, 709]

df = df[~df['BraTS21ID'].isin(to_exclude)]

# Work In Progress